<a href="https://colab.research.google.com/github/AdiKaipa/Session-9-Assignment-/blob/main/Learning_Phrase_representation_using_RNN_Encoder_Decoder_for_WikiQA_Dataset_(Model_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The WikiQA corpus: A set of publicly available pairs of questions and phrases collected and annotated for research on the answer to open-domain questions. In order to reflect the true information needs of general users, they used Bing query logs as a source of questions. Each question is linked to a Wikipedia page that potentially contains the answer

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
path_train = '/content/drive/MyDrive/END/END 8/WikiQA.tsv'

In [3]:
import pandas as pd
df_train = pd.read_csv(path_train, sep = '\t')

In [4]:
df_train.shape

(29208, 7)

In [5]:
df_train.head(30)

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0
5,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-5,"As such, African immigrants are to be distingu...",1
6,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
7,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0
8,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0
9,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1


In [6]:
df_train1 = df_train[df_train['Label'] == 1]

In [7]:
df_train1.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
5,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-5,"As such, African immigrants are to be distingu...",1
9,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
33,Q4,how a water pump works,D4,Pump,D4-4,Pumps operate by some mechanism (typically rec...,1
80,Q11,"how big is bmc software in houston, tx",D11,BMC Software,D11-3,"Employing over 6,000, BMC is often credited wi...",1
81,Q11,"how big is bmc software in houston, tx",D11,BMC Software,D11-4,"For 2011, the company recorded an annual reven...",1


In [8]:
df_data = df_train1[['Question', 'Sentence']]
df_data.shape

(1469, 2)

In [9]:

df_data.reset_index(inplace=True)
df_data.head()


,index,Question,Sentence
0,5,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,"As such, African immigrants are to be distingu..."
1,9,how are glacier caves formed?,A glacier cave is a cave formed within the ice...
2,33,how a water pump works,Pumps operate by some mechanism (typically rec...
3,80,"how big is bmc software in houston, tx","Employing over 6,000, BMC is often credited wi..."
4,81,"how big is bmc software in houston, tx","For 2011, the company recorded an annual reven..."


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data


from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [11]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [12]:
SRC = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TRG = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [13]:
fields = [('Question', SRC),('Sentence',TRG)]

In [14]:
example = [data.Example.fromlist([df_data.Question[i],df_data.Sentence[i]], fields) for i in range(df_data.shape[0])] 

In [15]:
QADataset = data.Dataset(example, fields)

In [16]:
vars(QADataset.examples[0])

{'Question': ['how',
  'african',
  'americans',
  'were',
  'immigrated',
  'to',
  'the',
  'us'],
 'Sentence': ['as',
  'such',
  ',',
  'african',
  'immigrants',
  'are',
  'to',
  'be',
  'distinguished',
  'from',
  'african',
  'american',
  'people',
  ',',
  'the',
  'latter',
  'of',
  'whom',
  'are',
  'descendants',
  'of',
  'mostly',
  'west',
  'and',
  'central',
  'africans',
  'who',
  'were',
  'involuntarily',
  'brought',
  'to',
  'the',
  'united',
  'states',
  'by',
  'means',
  'of',
  'the',
  'historic',
  'atlantic',
  'slave',
  'trade',
  '.']}

In [17]:
(train, valid) = QADataset.split(split_ratio=[0.90, 0.10], random_state=random.seed(SEED))

In [18]:
(len(train), len(valid))

(1322, 147)

In [19]:
SRC.build_vocab(train, min_freq = 2)
TRG.build_vocab(train, min_freq = 2)

In [20]:
print("size of SRC vocab: ", len(SRC.vocab))
print('size of TRG vocab: ', len(TRG.vocab))

size of SRC vocab:  808
size of TRG vocab:  3420


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
BATCH_SIZE = 128
train_iterator, valid_iterator = BucketIterator.splits((train, valid),sort = False, batch_size = BATCH_SIZE, device = device)

In [23]:
# Encoder
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, dropout):
    super().__init__()
    self.hid_dim = hid_dim
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, hid_dim)
    self.dropout = nn.Dropout(dropout)
  def forward(self, src):
    embedded = self.dropout(self.embedding(src))

    outputs, hidden = self.rnn(embedded)

       
    #outputs = [src len, batch size, hid dim * n directions]
    #hidden = [n layers * n directions, batch size, hid dim]
    return hidden

In [24]:
# Decoder 
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, dropout):
    super().__init__()
    self.hid_dim = hid_dim,
    self.output_dim = output_dim
    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
    self.fc = nn.Linear(emb_dim + hid_dim * 2, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, context):
    #input = [batch size]
    #hidden = [n layers * n directions, batch size, hid dim]
    #context = [n layers * n directions, batch size, hid dim]
        
    #n layers and n directions in the decoder will both always be 1, therefore:
    #hidden = [1, batch size, hid dim]
    #context = [1, batch size, hid dim]
    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))

    emb_con = torch.cat((embedded, context), dim = 2)
    output, hidden = self.rnn(emb_con, hidden)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
    output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)
    prediction = self.fc(output)
    
    return prediction, hidden
    


In [25]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    #assert encoder.hid_dim == decoder.hid_dim, \
    #  "Hidden dimensions of encoder and decoder must be equal"
  def forward(self, src, trg, teacher_forcing_ratio = 0.5):

    #src = [src len, batch size]
    #trg = [trg len, batch size]
    #teacher_forcing_ratio is probability to use teacher forcing
    #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

    batch_size = src.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
    context = self.encoder(src)
    hidden = context
    input = trg[0, :]
    for t in range(1, trg_len):
      output, hidden = self.decoder(input, hidden, context)
      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      input = trg[t] if teacher_force else top1
    
    return outputs


In [26]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [27]:
def init_weights(m):
  for name, param in m.named_parameters():
    nn.init.normal_(param.data, mean = 0.0, std = 0.01)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(808, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3420, 256)
    (rnn): GRU(768, 512)
    (fc): Linear(in_features=1280, out_features=3420, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [28]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,615,260 trainable parameters


In [29]:
# optimizer
optimizer = optim.Adam(model.parameters())

In [30]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [31]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0
  for i, batch in enumerate(iterator):
    src = batch.Question
    trg = batch.Sentence
    optimizer.zero_grad()
    output = model(src, trg)
    output_dim = output.shape[-1]
    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)
    loss = criterion(output, trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Sentence

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 5s
	Train Loss: 7.216 | Train PPL: 1361.673
	 Val. Loss: 5.218 |  Val. PPL: 184.534
Epoch: 02 | Time: 0m 4s
	Train Loss: 5.854 | Train PPL: 348.763
	 Val. Loss: 5.198 |  Val. PPL: 180.944
Epoch: 03 | Time: 0m 4s
	Train Loss: 5.759 | Train PPL: 316.908
	 Val. Loss: 5.115 |  Val. PPL: 166.494
Epoch: 04 | Time: 0m 4s
	Train Loss: 5.704 | Train PPL: 300.163
	 Val. Loss: 5.107 |  Val. PPL: 165.128
Epoch: 05 | Time: 0m 4s
	Train Loss: 5.643 | Train PPL: 282.281
	 Val. Loss: 5.074 |  Val. PPL: 159.852
Epoch: 06 | Time: 0m 4s
	Train Loss: 5.581 | Train PPL: 265.429
	 Val. Loss: 5.032 |  Val. PPL: 153.219
Epoch: 07 | Time: 0m 4s
	Train Loss: 5.521 | Train PPL: 249.875
	 Val. Loss: 5.045 |  Val. PPL: 155.203
Epoch: 08 | Time: 0m 4s
	Train Loss: 5.456 | Train PPL: 234.065
	 Val. Loss: 5.033 |  Val. PPL: 153.433
Epoch: 09 | Time: 0m 4s
	Train Loss: 5.425 | Train PPL: 227.044
	 Val. Loss: 5.082 |  Val. PPL: 161.042
Epoch: 10 | Time: 0m 4s
	Train Loss: 5.378 | Train PPL: 216.541